In [117]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm, trange

In [118]:
url = "http://citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&pN="

host = "localhost"
user = "root"
pw = ''
db = 'appartments'

num_pages = 6

In [119]:
headers = {
    'authority': 'www.citystar.ru',
    'cache-control': 'max-age=0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.106 Safari/537.36',
    'sec-fetch-dest': 'document',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'accept-language': 'ru,en;q=0.9',
}

session = requests.session()

In [120]:
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL server connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        # print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [121]:
connection = create_server_connection(host, user, pw)
execute_query(connection, "CREATE DATABASE IF NOT EXISTS appartments;")

connection = create_db_connection(host, user, pw, db)
execute_query(connection, """CREATE TABLE IF NOT EXISTS property (
    id INT AUTO_INCREMENT PRIMARY KEY,
    rooms INT,
    floor INT,
    floors_in_house INT,
    floor_relation FLOAT,
    area FLOAT,
    area_living FLOAT,
    area_kitchen FLOAT,
    price INT
);""")

MySQL server connection successful
MySQL Database connection successful


In [123]:
query = "DELETE FROM property;"
execute_query(connection, query)

query = "ALTER TABLE property AUTO_INCREMENT = 0;"
execute_query(connection, query)

pbar_1 = trange(1, num_pages+1)
for i in pbar_1:
    response = session.get(url+str(i), headers=headers)

    if response.status_code == 200:
        pbar_1.write("Success")
        # print()
    else:
        print("Bad result")

    soup = BeautifulSoup(response.content, 'html.parser')

    rows = soup.find_all('tr', class_='tbb')
    print(f"Получено записей: {len(rows)}")

    for r in tqdm(rows):
        try:
            params = r.find_all('td', class_='ttx')

            if ' ' in params[1].text:
                rooms = params[1].text.strip().split()[0]
            else:
                rooms = params[1].text.strip()
            
            if rooms[0] == 'О':
                rooms = 1
            elif rooms[0] == 'Д':
                rooms = 2
            elif rooms[0] == 'Т':
                rooms = 3
            elif rooms[0] == 'Ч':
                rooms = 4
            elif rooms[0] == 'П':
                rooms = 5
            elif rooms[0] == 'Ш':
                rooms = 6
            else:
                print('WOW!')

            floors = list(map(int, params[4].text.strip().split('/')))

            query = f"INSERT INTO property (rooms, floor, floors_in_house, floor_relation, " \
                + f"area, area_living, area_kitchen, price) VALUES ({rooms}, {floors[0]}, " \
                + f"{floors[1]}, {(floors[0] - 1) / (floors[1] - 1)}, {params[5].text}, " \
                + f"{params[6].text}, {params[7].text}, {params[9].text});"

            execute_query(connection, query)

        except Exception as error:
            print("An exception occurred:", error)

  0%|          | 0/6 [00:00<?, ?it/s]

Success
Получено записей: 102


  0%|          | 0/102 [00:00<?, ?it/s]

An exception occurred: list index out of range
An exception occurred: list index out of range
An exception occurred: list index out of range
An exception occurred: list index out of range
WOW!
Error: '1054 (42S22): Unknown column 'Многокомнатная' in 'field list''
An exception occurred: division by zero
An exception occurred: division by zero
An exception occurred: division by zero
Success
Получено записей: 102


  0%|          | 0/102 [00:00<?, ?it/s]

...

An exception occurred: list index out of range
An exception occurred: division by zero
An exception occurred: list index out of range
WOW!
Error: '1054 (42S22): Unknown column 'Многокомнатная' in 'field list''
WOW!
Error: '1054 (42S22): Unknown column 'Многокомнатная' in 'field list''
An exception occurred: division by zero
An exception occurred: division by zero
An exception occurred: division by zero
An exception occurred: division by zero
An exception occurred: division by zero
WOW!
An exception occurred: division by zero
Success
Получено записей: 102


  0%|          | 0/102 [00:00<?, ?it/s]

An exception occurred: list index out of range
An exception occurred: division by zero
An exception occurred: division by zero
WOW!
Error: '1054 (42S22): Unknown column 'Многокомнатная' in 'field list''
An exception occurred: list index out of range
An exception occurred: division by zero
WOW!
An exception occurred: division by zero
An exception occurred: division by zero
An exception occurred: list index out of range
An exception occurred: division by zero
Success
Получено записей: 33


  0%|          | 0/33 [00:00<?, ?it/s]

An exception occurred: list index out of range
WOW!
Error: '1054 (42S22): Unknown column 'Многокомнатная' in 'field list''
WOW!
Error: '1054 (42S22): Unknown column 'Многокомнатная' in 'field list''
An exception occurred: division by zero
An exception occurred: list index out of range
An exception occurred: division by zero
